In [1]:
import pandas as pd

data = pd.read_json("./05_clean_data.json.zip")
data

,num_vars,num_clauses,baseline_alg,baseline_alg_conf,runtime_baseline,baseline_sample_size,path_to_baseline_sample,parameters,instance_name,lower_bound,upper_bound,iteration_info,time_used_by_yasa,timelimit_for_samplns,samplns_used,runtime_samplns,time_limit
0,1272,2980,YASA,t2_m1_null,3.389,71,2023-03-01_13-51-03/43_1_7_2_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",integrator_arm7,27,47,"[{'nbrhd_tuples': 92, 'nbrhd_confs': 4, 'itera...",3.389,896.611,True,901.019550,900
1,96,183,YASA,t2_m1_null,0.252,17,2023-03-01_13-51-03/18_1_7_2_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",axTLS,10,11,"[{'nbrhd_tuples': 208, 'nbrhd_confs': 7, 'iter...",0.252,899.748,True,924.424374,900
2,9,16,YASA,t2_m1_null,0.202,8,2023-03-01_13-51-03/2_1_7_3_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",lcm,6,6,"[{'nbrhd_tuples': 37, 'nbrhd_confs': 8, 'itera...",0.202,899.798,True,0.104894,900
3,230,1181,YASA,t2_m1_null,0.982,246,2023-03-01_13-51-03/24_1_7_5_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",fiasco_2017-09-26_11-30-56,223,225,"[{'nbrhd_tuples': 224, 'nbrhd_confs': 20, 'ite...",0.982,899.018,True,928.357737,900
4,263,1699,YASA,t2_m1_null,1.483,510,2023-03-01_13-51-03/26_1_7_3_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",uclibc_2008-06-05_13-46-47,505,505,"[{'nbrhd_tuples': 248, 'nbrhd_confs': 49, 'ite...",1.483,898.517,True,847.626329,900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,16,33,YASA,t2_m1_null,0.205,7,2023-03-01_13-51-03/6_1_7_4_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",car,5,5,"[{'nbrhd_tuples': 77, 'nbrhd_confs': 7, 'itera...",0.205,899.795,True,0.065082,900
226,10,17,YASA,t2_m1_null,0.202,7,2023-03-01_13-51-03/3_1_7_4_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",email,6,6,"[{'nbrhd_tuples': 70, 'nbrhd_confs': 7, 'itera...",0.202,899.798,True,0.197271,900
227,334,92,YASA,t2_m1_null,0.478,20,2023-03-01_13-51-03/29_1_7_2_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",toybox_2020-12-06_00-02-46,7,14,"[{'nbrhd_tuples': 139, 'nbrhd_confs': 2, 'iter...",0.478,899.522,True,920.295881,900
228,1229,2875,YASA,t2_m1_null,3.433,77,2023-03-01_13-51-03/41_1_7_2_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",atlas_mips32_4kc,32,45,"[{'nbrhd_tuples': 229, 'nbrhd_confs': 11, 'ite...",3.433,896.567,True,926.051666,900


In [2]:
# calculate the best known lower bound
best_lower_bound = data.groupby(["instance_name"])["lower_bound"].max().reset_index().rename(columns={"lower_bound": "best_lower_bound"})
data = data.merge(best_lower_bound, on="instance_name")

In [3]:
# calculate the best upper bound
best_upper_bound = data.groupby(["instance_name"])["upper_bound"].min().reset_index().rename(columns={"upper_bound": "best_upper_bound"})
data = data.merge(best_upper_bound, on="instance_name")

In [4]:
# calculate best baseline algorithm sample
baseline_data = pd.read_json("./00_simple_baseline_data.json.zip")
best_baseline = baseline_data.groupby(["instance_name"])[["sample_size"]].min().reset_index().rename(columns={"sample_size": "best_baseline_sample_size"})
data = data.merge(best_baseline, on="instance_name", how="left")


In [5]:
t = data.groupby("instance_name")[["num_vars", "num_clauses", "best_baseline_sample_size", "baseline_sample_size", "upper_bound", "lower_bound", "best_upper_bound", "best_lower_bound"]].mean()
t.sort_values(["num_vars", "num_clauses"], inplace=True)
# checking consistency is important.
assert (t["best_lower_bound"]<=t["best_baseline_sample_size"]).all()
assert (t["best_upper_bound"]>=t["best_lower_bound"]).all()
t

,num_vars,num_clauses,best_baseline_sample_size,baseline_sample_size,upper_bound,lower_bound,best_upper_bound,best_lower_bound
instance_name,,,,,,,,
calculate,9.0,15.0,9.0,10.0,5.0,5.0,5.0,5.0
lcm,9.0,16.0,8.0,8.0,6.0,6.0,6.0,6.0
email,10.0,17.0,6.0,7.0,6.0,6.0,6.0,6.0
ChatClient,14.0,20.0,8.0,8.0,7.0,7.0,7.0,7.0
toybox_2006-10-31_23-30-06,16.0,13.0,9.0,10.0,8.0,8.0,8.0,8.0
car,16.0,33.0,6.0,7.0,5.0,5.0,5.0,5.0
FeatureIDE,19.0,27.0,11.0,12.0,8.0,7.0,8.0,7.0
FameDB,22.0,40.0,8.0,8.0,8.0,8.0,8.0,8.0
APL,23.0,35.0,9.0,10.0,7.0,7.0,7.0,7.0
